In [4]:
%store -r params
%store -r secrets

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import to_avro, from_avro

In [3]:
spark = SparkSession.builder.appName("Ingestion").getOrCreate()

24/03/31 14:00:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
read_checkpoint_location = params["checkpoints"]["ingestion_read"]
os.makedirs(read_checkpoint_location,exist_ok=True)
write_checkpoint_location = params["checkpoints"]["ingestion_write"]
os.makedirs(write_checkpoint_location,exist_ok=True)

with open(params["avro_schema_file"]) as f:
    avsc = f.read()

In [8]:
avro_df = (spark.readStream
          .format("kafka")
          .option("kafka.bootstrap.servers", secrets["GREEN_KAFKA_SERVERS"])
          .option("kafka.sasl.mechanism", "PLAIN")
          .option("kafka.security.protocol", "SASL_SSL")
          .option("kafka.sasl.jaas.config", "org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(secrets["GREEN_KAFKA_USERNAME"], secrets["GREEN_KAFKA_PASSWORD"]))
          .option("subscribe", params["topic"]["green_landing"])
          .option("kafka.group.id", "ingestion-engine")
          .option('checkpointLocation', read_checkpoint_location)
          .load()
        )

24/03/31 14:05:44 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    


In [9]:
json_df = avro_df.withColumn("value", from_avro("value", avsc))

In [10]:
def upsert(json_table_df, batch_id):
    json_table_df.createOrReplaceTempView("ingestion_temp_view")
    merge_statement = """MERGE INTO creditcard_prod.card_details_raw as Target
                         USING ingestion_temp_view as Source
                         ON Source.value == Target.value
                         WHEN MATCHED THEN
                         UPDATE SET *
                         WHEN NOT MATCHED THEN
                         INSERT *
                    """
    json_table_df.sparkSession.sql(merge_statement)


In [11]:
sQuery = (json_df.writeStream
          .queryName("Ingestion")
          .foreachBatch(upsert)
          .option("checkpointLocation",write_checkpoint_location)
          .outputMode("append")
          .start()
)


24/03/31 14:05:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/31 14:05:59 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    
24/03/31 14:05:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, group.id, auto.offset.reset]' we

In [ ]:
#sQuery = (json_df.writeStream
#          .queryName("Ingestion")
#          .option("checkpointLocation",write_checkpoint_location)
#          .outputMode("append")
#          .toTable("creditcard_prod.card_details_raw")
#)